In [20]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.2f}".format)


In [21]:
df = pd.read_excel("top_30_mutual_funds_excel.xlsx")


In [22]:
df = df.rename(columns={
    "risk-adjusted return score": "risk_adjusted_return_score",
    "cost efficiency score": "cost_efficiency_score",
    "consistency score": "consistency_score",
    "fund stability": "fund_stability"
})


In [23]:
required_cols = [
    "scheme_name", "returns_5yr", "sharpe", "standard_deviation",
    "risk_bucket", "category",
    "cost_efficiency_score", "consistency_score", "fund_stability"
]

missing = set(required_cols) - set(df.columns)
assert not missing, f"Missing columns: {missing}"


In [24]:
num_features = [
    "returns_5yr",
    "sharpe",
    "standard_deviation",
    "cost_efficiency_score",
    "consistency_score",
    "fund_stability"
]


In [25]:
def zscore(series):
    return (series - series.mean()) / series.std()


In [26]:
df["z_returns_5yr"] = zscore(df["returns_5yr"])
df["z_sharpe"] = zscore(df["sharpe"])
df["z_std_dev"] = zscore(df["standard_deviation"]) * -1  # lower risk = better


In [27]:
df["target_score_z"] = (
    0.5 * df["z_returns_5yr"] +
    0.3 * df["z_sharpe"] +
    0.2 * df["z_std_dev"]
)


In [28]:
df[[
    "z_returns_5yr",
    "z_sharpe",
    "z_std_dev",
    "target_score_z"
]].corr()


,z_returns_5yr,z_sharpe,z_std_dev,target_score_z
z_returns_5yr,1.00,-0.00,-0.49,0.77
z_sharpe,-0.00,1.00,-0.08,0.54
z_std_dev,-0.49,-0.08,1.00,-0.13
target_score_z,0.77,0.54,-0.13,1.00


In [29]:
risk_map = {
    "Low": ["Low Risk"],
    "Moderate": ["Low Risk", "Moderate", "Moderately Low"],
    "High": ["Moderate", "High Risk"]
}


In [30]:
return_map = {
    "Low": (0, 10),
    "Moderate": (8, 18),
    "High": (15, 30)
}


In [31]:
def filter_by_risk(df, user_risk):
    return df[df["risk_bucket"].isin(risk_map[user_risk])]


In [32]:
def filter_by_return_band(df, band):
    low, high = return_map[band]
    return df[(df["returns_5yr"] >= low) & (df["returns_5yr"] <= high)]


In [33]:
def recommend_top_funds(df, user_risk, return_band, top_n=5):
    filtered = df.copy()

    filtered = filter_by_risk(filtered, user_risk)
    filtered = filter_by_return_band(filtered, return_band)

    # Fallback: never return empty
    if filtered.empty:
        filtered = df.copy()

    filtered = filtered.sort_values(
        "target_score_z", ascending=False
    )

    return filtered[[
        "scheme_name",
        "target_score_z",
        "returns_5yr",
        "risk_bucket"
    ]].head(top_n)


In [34]:
def project_amount(amount, rate, years, mode="SIP"):
    r = rate / 100

    if mode == "SIP":
        months = years * 12
        monthly_rate = r / 12
        return amount * (
            ((1 + monthly_rate) ** months - 1) / monthly_rate
        ) * (1 + monthly_rate)

    else:  # Lumpsum
        return amount * ((1 + r) ** years)


In [35]:
def attach_projection(df, investment_type, amount, years):
    df = df.copy()
    df["projected_value"] = df["returns_5yr"].apply(
        lambda r: project_amount(amount, r, years, investment_type)
    )
    return df


In [36]:
top_funds = recommend_top_funds(
    df,
    user_risk="Moderate",
    return_band="Moderate",
    top_n=5
)


In [37]:
projection = attach_projection(
    top_funds,
    investment_type="SIP",
    amount=5000,
    years=10
)


In [38]:
projection


,scheme_name,target_score_z,returns_5yr,risk_bucket,projected_value
5,AXIS FTP – Series 104 – 1112Days,0.54,14.00,Low Risk,1310456.91
1,Kotak Multi Asset Allocator FoF – Dynamic – Di...,0.13,15.30,Moderate,1419330.14
14,ICICI Pru Asset Allocator Fund,-0.06,12.80,Moderately Low,1218647.18
